# Elastic Search

There are different ways to interact with Elastic Search:

- cURL
- Requests
- The `elasticsearch` package/library
- Batch/Bulk Processing

## Requests

In [3]:
import os
import requests
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
load_dotenv()

elastic_user = os.getenv("ELASTIC_USER")
elastic_password = os.getenv("ELASIC_PASSWORD")

```powershell
# Example: GET /_cat/nodes?v -> get all nodes
curl.exe --cacert config\certs\http_ca.crt -u "$($Env:ELASTIC_USER):$($Env:ELASTIC_PASSWORD)" --insecure -X GET "https://localhost:9200/_cat/nodes?v" --noproxy localhost
```

In [19]:
# Define the URL
url = "https://localhost:9200/_cat/nodes?v"

# Make the GET request
response = requests.get(
    url,
    auth=HTTPBasicAuth(elastic_user, elastic_password),
    #verify=False,  # Disable SSL verification
    verify="C:\\Users\\msagardia\\packages\\elasticsearch-8.14.3\\config\\certs\\http_ca.crt",
    proxies={"http": None, "https": None}  # Bypass proxy
)

# Print the response
print(response.text)

ip        heap.percent ram.percent cpu load_1m load_5m load_15m node.role   master name
127.0.0.1           20          97   3                          cdfhilmrstw *      RDES009A



```powershell
# Example: GET /_cat/indices?v -> list all indices
curl.exe --cacert config\certs\http_ca.crt -u "$($Env:ELASTIC_USER):$($Env:ELASTIC_PASSWORD)" --insecure -X GET "https://localhost:9200/_cat/indices?v" --noproxy localhost
```

In [20]:
# Define the URL
url = "https://localhost:9200/_cat/indices?v"

# Make the GET request
response = requests.get(
    url,
    auth=HTTPBasicAuth(elastic_user, elastic_password),
    #verify=False,  # Disable SSL verification
    verify="C:\\Users\\msagardia\\packages\\elasticsearch-8.14.3\\config\\certs\\http_ca.crt",
    proxies={"http": None, "https": None}  # Bypass proxy
)

# Print the response
print(response.text)

health status index                                                              uuid                   pri rep docs.count docs.deleted store.size pri.store.size dataset.size
green  open   .internal.alerts-transform.health.alerts-default-000001            P7VMqfVWRre9OapmMd36rg   1   0          0            0       249b           249b         249b
green  open   .internal.alerts-observability.logs.alerts-default-000001          ZGmcHDzuSBaYLTKgLqjwPw   1   0          0            0       249b           249b         249b
green  open   .internal.alerts-observability.uptime.alerts-default-000001        G615buOwQhCU_WGl-jmMsQ   1   0          0            0       249b           249b         249b
green  open   .internal.alerts-ml.anomaly-detection.alerts-default-000001        JyFHy8-mTuKUzYhcEE7LPA   1   0          0            0       249b           249b         249b
green  open   .internal.alerts-observability.slo.alerts-default-000001           yrBABNDUTMmyzhlKS9teNQ   1   0          0   

## Elastic Search Package

In [36]:
import os
from dotenv import load_dotenv
load_dotenv()

elastic_user = os.getenv("ELASTIC_USER")
elastic_password = os.getenv("ELASIC_PASSWORD")

In [37]:
import warnings
from urllib3.exceptions import InsecureRequestWarning

# Suppress only the specific InsecureRequestWarning
warnings.simplefilter('ignore', InsecureRequestWarning)

In [38]:
from elasticsearch import Elasticsearch

# Create an instance of the Elasticsearch client
es = Elasticsearch(
    ['https://localhost:9200'],
    basic_auth=(elastic_user, elastic_password),
    verify_certs=False  # This disables SSL verification, similar to --insecure
)

c:\Users\A200239740\AppData\Local\anaconda3\envs\elastic\lib\site-packages\elasticsearch\_sync\client\__init__.py:399: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [40]:
# Make a GET request to /_cat/nodes?v
response = es.cat.nodes(format="json")

# Print the response
print(response)

[{'ip': '127.0.0.1', 'heap.percent': '42', 'ram.percent': '97', 'cpu': '3', 'load_1m': None, 'load_5m': None, 'load_15m': None, 'node.role': 'cdfhilmrstw', 'master': '*', 'name': 'RDES009A'}]


In [41]:
# Make a GET request to /_cat/indices?v
response = es.cat.indices(format="json")

# Print the response
print(response)

[{'health': 'green', 'status': 'open', 'index': '.internal.alerts-transform.health.alerts-default-000001', 'uuid': 'P7VMqfVWRre9OapmMd36rg', 'pri': '1', 'rep': '0', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '249b', 'pri.store.size': '249b', 'dataset.size': '249b'}, {'health': 'green', 'status': 'open', 'index': '.internal.alerts-observability.logs.alerts-default-000001', 'uuid': 'ZGmcHDzuSBaYLTKgLqjwPw', 'pri': '1', 'rep': '0', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '249b', 'pri.store.size': '249b', 'dataset.size': '249b'}, {'health': 'green', 'status': 'open', 'index': '.internal.alerts-observability.uptime.alerts-default-000001', 'uuid': 'G615buOwQhCU_WGl-jmMsQ', 'pri': '1', 'rep': '0', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '249b', 'pri.store.size': '249b', 'dataset.size': '249b'}, {'health': 'green', 'status': 'open', 'index': '.internal.alerts-ml.anomaly-detection.alerts-default-000001', 'uuid': 'JyFHy8-mTuKUzYhcEE7LPA', 'pri': '1', 'rep':

## Batch/Bulk Processing

In this section, the file [`products-bulk.json`](./products-bulk.json) with 1,000 dummy product documents is ingested.

In [17]:
import os
import requests
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
load_dotenv()

elastic_user = os.getenv("ELASTIC_USER")
elastic_password = os.getenv("ELASIC_PASSWORD")
elastic_home = os.getenv("ELASTIC_HOME")

# Verify that the environment variables are correctly set
if not all([elastic_user, elastic_password, elastic_home]):
    raise ValueError("One or more environment variables are not set")

`cURL` commands to ingest [`products-bulk.json`](./products-bulk.json) with and without certificate:

```bash
# Set variables in Powershell for easier and more secure use.
# To use them: $Env:ELASTIC_USER
$Env:ELASTIC_USER = "elastic"
$Env:ELASTIC_PASSWORD = "..."
$Env:ELASTIC_HOME = "..."
# Bash. To use them: $ELASTIC_USER
export ELASTIC_USER="elastic"
export ELASTIC_PASSWORD="..."
export ELASTIC_HOME="..."

# Without CA certificate validation.
# This is fine for development clusters, but don't do this in production!
# "@products-bulk.json" means we ingest a file, not a path
curl --insecure -u $ELASTIC_USER:$ELASIC_PASSWORD -H "Content-Type:application/x-ndjson" -XPOST https://localhost:9200/products/_bulk --data-binary "@products-bulk.json"
# Windows
curl.exe --insecure -u "$($Env:ELASTIC_USER):$($Env:ELASTIC_PASSWORD)" -H "Content-Type:application/x-ndjson" -XPOST https://localhost:9200/products/_bulk --data-binary "@products-bulk.json"  --noproxy localhost

# With CA certificate validation. 
# The certificate is located at $ELASTIC_HOME/config/certs/http_ca.crt
# "@products-bulk.json" means we ingest a file, not a path
curl --cacert $ELASTIC_HOME/config/certs/http_ca.crt -u $ELASTIC_USER:$ELASIC_PASSWORD -H "Content-Type:application/x-ndjson" -XPOST https://localhost:9200/products/_bulk --data-binary "@products-bulk.json"
# Windows
curl.exe --cacert "$($Env:ELASTIC_HOME)\config\certs\http_ca.crt" -u "$($Env:ELASTIC_USER):$($Env:ELASTIC_PASSWORD)" -H "Content-Type:application/x-ndjson" -XPOST https://localhost:9200/products/_bulk --data-binary "@products-bulk.json" --noproxy localhost
```

In [16]:
# Define the URL
url = "https://localhost:9200/products/_bulk"

# Read the data from the file
with open("products-bulk.json", "rb") as data_file:
    data = data_file.read()

# Make the POST request
response = requests.post(
    url,
    auth=HTTPBasicAuth(elastic_user, elastic_password),
    headers={"Content-Type": "application/x-ndjson"},
    data=data,
    verify=os.path.join(elastic_home, "config", "certs", "http_ca.crt"),
    proxies={"http": None, "https": None}  # Bypass proxy
)

# Print the response
print(response.text) # {"errors":false,"took":155,"items":[{"index":{"_index":"products","_id":"...

{"errors":false,"took":155,"items":[{"index":{"_index":"products","_id":"1","_version":3,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":1068,"_primary_term":1,"status":200}},{"index":{"_index":"products","_id":"2","_version":3,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":1069,"_primary_term":1,"status":200}},{"index":{"_index":"products","_id":"3","_version":3,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":1070,"_primary_term":1,"status":200}},{"index":{"_index":"products","_id":"4","_version":3,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":945,"_primary_term":1,"status":200}},{"index":{"_index":"products","_id":"5","_version":3,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":1071,"_primary_term":1,"status":200}},{"index":{"_index":"products","_id":"6","_version":3,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq

```